## Advent of Code 2023 Day 3 ⚙️ 🚡 ⚙️

When I saw the input data I was concerned that the new tool in my kit, FParsec would be useless.

As it happens, there is the `getPosition` parser, which can insert the current row and column of the input stream into the parsed result. Very handy, and just what I needed for locating all of these gears and whatnot!

In [76]:
#r "nuget:FParsec"
open FParsec

let loadFile fname = System.IO.File.ReadAllText(fname)

let sample = "day3.sample.txt"
let data = "day3.data.txt"

let str = pstring

Installed Packages FParsec, 1.1.1

In [77]:
let numPositionParser = tuple3 getPosition pint32 getPosition
let skipNonDigit = skipManySatisfy (Char.IsDigit >> not)
let enginePartParser = skipNonDigit >>. numPositionParser .>> skipNonDigit
let fileParser = many enginePartParser

let parseEngineParts (s: string) =
    match run fileParser s with
    | Success (result, _, _) -> result
    | Failure (message, _, _) -> failwith message


In [78]:
type Number = {
    Number : int
    Row : int
    Col : int
    Length : int
}


module Number =
    let fromParsedData (startPos : Position, num : int, endPos : Position) : Number =
        {
            Number = num
            Row = int startPos.Line - 1
            Col = int startPos.Column - 1
            Length = int endPos.Column - int startPos.Column
        }

    let neighborPositions (dataRows : string array) (n : Number) =
        let w = dataRows[0].Length
        let h = dataRows.Length
        [
            // left neighbor
            (n.Col - 1, n.Row)
            // right neighbor
            (n.Col + n.Length, n.Row)
            // rowAbove
            yield! seq {(n.Col - 1)..(n.Col+n.Length)} |> Seq.map (fun c -> (c, n.Row-1))
            // rowBelow
            yield! seq {(n.Col - 1)..(n.Col+n.Length)} |> Seq.map (fun c -> (c, n.Row+1))
        ]
        |> List.filter (fun (col, row) ->
            row >= 0 && row + 1 <= w && col >= 0 && col + 1 <= h
            )

    let isEnginePart (dataRows : string array) (n : Number) =
        neighborPositions dataRows n
        |> List.map (fun (col, row) -> dataRows[row][col])
        |> List.exists ((<>) '.')

    let neighboringGear (dataRows : string array) (n : Number) =
        neighborPositions dataRows n
        |> List.tryFind (fun (col, row) -> dataRows[row][col] = '*')



In [79]:
display "Part 1 Sample answer"

let testData = loadFile(sample)
let parseResult = parseEngineParts testData
let numbers =
    parseResult
    |> List.map Number.fromParsedData
let dataRows = testData.Split('\n')

numbers
|> List.filter (Number.isEnginePart dataRows)
|> List.sumBy (fun x -> x.Number)


Part 1 Sample answer

4361

In [80]:
display "Part 1 Real answer"

let testData = loadFile(data)
let parseResult = parseEngineParts testData
let numbers =
    parseResult
    |> List.map Number.fromParsedData
let dataRows = testData.Split('\n')

numbers
|> List.filter (Number.isEnginePart dataRows)
|> List.sumBy (fun x -> x.Number)

Part 1 Real answer

553079

In [99]:
display "Part 2 answer"

let realData = loadFile(data)
let parseResult = parseEngineParts realData
let numbers =
    parseResult
    |> List.map Number.fromParsedData
let dataRows = realData.Split('\n')


numbers
|> List.map (fun n -> (n, Number.neighboringGear dataRows n))
|> List.filter (snd >> Option.isSome)
|> List.groupBy snd
|> List.filter (fun x -> (snd x) |> List.length = 2)
|> List.map (fun x ->
    (snd x) |> List.map fst |> List.map (fun y -> y.Number) |> List.reduce (*))
|> List.sum



Part 2 answer

84363105